# Re-ranking RAG

Re-ranking is an advanced technique used to improve the relevance and quality of retrieved documents before they are fed into the language model for generation. It's typically applied after the initial retrieval step but before the generation step.

## Here's a detailed explanation with examples:

## Basic Concept:

In a typical RAG system, documents are initially retrieved based on their similarity to the query, often using vector similarity search. Re-ranking takes these initially retrieved documents and applies additional criteria or more sophisticated algorithms to reorder them, aiming to bring the most relevant documents to the top.

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0 \
     flashrank==0.2.8 \
     faiss_cpu==1.8.0.post1

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
# Import Libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import (
    LLMChainExtractor,
    EmbeddingsFilter,
)
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
import getpass
import os

# Provide Google API Key. You can create Google API key at following lin

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following lin

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    # Iterate through each document and format the output
    for i, d in enumerate(docs):
        print(f"{'-' * 50}\nDocument {i + 1}:")
        print(f"Content:\n{d.page_content}\n")
        print("Metadata:")
        for key, value in d.metadata.items():
            print(f"  {key}: {value}")
    print(f"{'-' * 50}")  # Final separator for clarity

# Example usage
# Assuming `docs` is a list of Document objects


In [ ]:
# Import necessary libraries
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from a web URL
documents = WebBaseLoader("https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt").load()

# Split documents into chunks of 500 characters with 100 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Add unique IDs to each text chunk
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

# Create embeddings for the text chunks
embedding = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})

# Initialize a FAISS retriever with the text embeddings
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 20})

# Define a query and retrieve relevant documents
query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.invoke(query)
#print(docs)
# Print the retrieved documents
pretty_print_docs(docs)

--------------------------------------------------
Document 1:
Content:
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 
And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

Metadata:
  source: https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt
  title: chroma-langchain/state_of_the_union.txt at master · hwchase17/chroma-langchain · GitHub
  description: Contribute to hwchase17/chroma-langchain development by creating an account on GitHub.
  language: en
  id: 88
--------------------------------------------------
Document 2:
Content:
And tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. 
By the end of this year, the deficit will be down to less than half what it was before I took

# Reranking with FlashRank

We will wrap our base retriever with a ContextualCompressionRetriever that uses FlashrankRerank as its compressor.


In [ ]:
# Import necessary classes for contextual compression and document retrieval
from langchain.retrievers import ContextualCompressionRetriever
# Create re-ranker
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank

# Initialize the language model with specified settings
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Set up the document compressor using FlashRank
compressor = FlashrankRerank()

# Create a compression retriever that uses the base retriever and compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Invoke the compression retriever with a specific query
compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)

# Print the IDs of the compressed documents retrieved
print([doc.metadata["id"] for doc in compressed_docs])

[88, 65, 50]


In [ ]:
pretty_print_docs(compressed_docs)

--------------------------------------------------
Document 1:
Content:
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 
And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

Metadata:
  source: https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt
  title: chroma-langchain/state_of_the_union.txt at master · hwchase17/chroma-langchain · GitHub
  description: Contribute to hwchase17/chroma-langchain development by creating an account on GitHub.
  language: en
  id: 88
  relevance_score: 0.9987671971321106
--------------------------------------------------
Document 2:
Content:
And tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. 
By the end of this year, the deficit will be down to le

In [ ]:
# Import the RetrievalQA chain for question-answering tasks
from langchain.chains import RetrievalQA

# Create a RetrievalQA chain using the language model and the compression retriever
chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

# Invoke the chain with a specific query to get a response
reponse = chain.invoke(query)

# Print the result of the response
print(reponse["result"])

The president said that he nominated Ketanji Brown Jackson to the United States Supreme Court 4 days ago. He referred to her as one of our nation's top legal minds who will continue Justice Breyer's legacy of excellence. 

